In [2]:
import ee
import pandas as pd
import time

ee.Authenticate()


Successfully saved authorization token.


In [3]:
# Initialize the Earth Engine module.
ee.Initialize()

df = pd.read_csv('../data/label_unscaled.csv')
df['test'] =False

In [4]:
import os
import re

def extract_info_from_filename(filename):
    # Regular expression to match the pattern
    pattern = r'(\d{4})_(\d+\.\d+)_(\d+\.\d+)_(\d+\.\d+).tfrecord.gz'

    # Search for matches using the pattern
    match = re.search(pattern, filename)

    if match:
        year = match.group(1)
        longtitude = match.group(2)
        latitude = match.group(3)
        return int(year), float(longtitude), float(latitude)
    else:
        return None

# Specify the directory path
directory_path = '../data/raw_records/test/'

# List all entries in the directory and filter only files
file_names = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

# Updated loop with a check for None
for file_name in file_names:
    info = extract_info_from_filename(os.path.join(directory_path, file_name))
    
    if info is not None:
        year, longitude, latitude = info
        # Use .loc to update the 'test' column for matching rows
        df.loc[(df['centroid_latitude'] == latitude) & (df['centroid_longtitude'] == longitude), 'test'] = True


In [5]:
df = df[df['test']]

In [6]:
df

,year,cell_id,centroid_longtitude,centroid_latitude,wi,test
1,2014,"POLYGON ((226313.1598511024 2458714.686800875,...",102.311955,22.242135,0.744735,True
2,2014,"POLYGON ((226313.1598511024 2465464.686800875,...",102.310790,22.303049,0.744735,True
9,2014,"POLYGON ((239813.1598511024 2465464.686800875,...",102.441720,22.305168,0.747482,True
12,2014,"POLYGON ((246563.1598511024 2445214.686800875,...",102.510420,22.123417,1.821595,True
16,2014,"POLYGON ((246563.1598511024 2472214.686800875,...",102.506108,22.367111,0.999738,True
...,...,...,...,...,...,...
13099,2019,"POLYGON ((955313.1598511024 1405714.686800875,...",109.159535,12.713695,2.714625,True
13108,2019,"POLYGON ((955313.1598511024 1466464.686800875,...",109.168677,13.261581,2.538603,True
13110,2019,"POLYGON ((955313.1598511024 1479964.686800875,...",109.170766,13.383330,2.591136,True
13132,2019,"POLYGON ((962063.1598511024 1459714.686800875,...",109.229768,13.199686,2.569571,True


In [7]:
PREFIX = 'asset-index-3'
SCALE = 30  # meters
KSIZE = 112  # half the size of the desired patch in pixels (225 / 2)
BANDS = ['B2', 'B3', 'B4', 'B8', 'B11','B12']
SCALE_FACTOR = 10000  # Scaling factor for Sentinel-2 reflectance values
DATASET = 'COPERNICUS/S2'
DYNAMIC = 'GOOGLE/DYNAMICWORLD/V1'
NUM_BATCH = 20
BATCH_SIZE = len(df) // NUM_BATCH

def get_quarter_dates(year):
    # Returns a list of date ranges for each quarter
    return [
        (f'{year}-01-01', f'{year}-03-31'), # Q1
        (f'{year}-04-01', f'{year}-06-30'), # Q2
        (f'{year}-07-01', f'{year}-09-30'), # Q3
        (f'{year}-10-01', f'{year}-12-31')  # Q4
    ]

def export_image(longtitude, latitude, year, wi):

    # Define the point of interest and create a buffer region around it
    point = ee.Geometry.Point([longtitude, latitude])
    buffered_point = point.buffer(SCALE * KSIZE)

    # Function to estimate cloud coverage over the ROI
    def cloudiness(image):
        qa_band = image.select('QA60')
        opaque_clouds = qa_band.bitwiseAnd(1 << 10).neq(0)
        cirrus_clouds = qa_band.bitwiseAnd(1 << 11).neq(0)
        total_cloud_mask = opaque_clouds.Or(cirrus_clouds)
        cloud_coverage_ratio = total_cloud_mask.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=buffered_point,
            scale=SCALE
        ).get('QA60')
        return image.set('cloudiness', cloud_coverage_ratio)
    
    quarterly_images = []

     # Retrieve the least cloudy image of the year as a fallback
    yearly_collection = ee.ImageCollection(DATASET) \
        .filterDate(f'{year}-01-01', f'{year}-12-31') \
        .filterBounds(buffered_point) \
        .map(cloudiness)

    yearly_least_cloudy = yearly_collection.sort('cloudiness').first().divide(SCALE_FACTOR)

    for i, (start_date, end_date) in enumerate(get_quarter_dates(year)):
        collection = ee.ImageCollection(DATASET) \
            .filterDate(start_date, end_date) \
            .filterBounds(buffered_point) \
            .map(cloudiness)
        
        # Check if the collection is empty
        count = collection.size().getInfo()
        if count == 0:
            least_cloudy_image = yearly_least_cloudy
        else:
            least_cloudy_image = collection.sort('cloudiness').first().divide(SCALE_FACTOR)

        quarter_suffix = f'_{i+1}'
        renamed_bands = [band + quarter_suffix for band in BANDS]
        least_cloudy_image = least_cloudy_image.select(BANDS, renamed_bands)


        quarterly_images.append(least_cloudy_image)
    
    # Yearly Dynamic World collection filtered by date and bounds
    # Compute the median of the 'label' band over the year
    # Rename band into landcover
    dwYearly = ee.ImageCollection(DYNAMIC) \
        .filterDate(f'{year}-01-01', f'{year}-12-31') \
        .filterBounds(buffered_point) \
        .select('label') \
        .median()
    
    quarterly_images.append(dwYearly)

    # Combine the quarterly images into a single image
    combined_image = ee.Image.cat(quarterly_images)
    
    # Define the point of interest as a feature collection
    points = ee.FeatureCollection([ee.Feature(point, {'longtitude': longtitude, 'latitude': latitude})])

        # Set up the export task
    task = get_array_patches(
        combined_image,
        points,
        longtitude,
        latitude,
        year,
        wi
    )
    # Start the export
    task.start()

# Function to get array patches
def get_array_patches(img, points, longtitude, latitude, year, wi):
    kern = ee.Kernel.square(radius=KSIZE, units='pixels')
    patches_array = img.neighborhoodToArray(kern)
    samples = points.map(lambda pt: sample_patch(pt, patches_array, longtitude, latitude, wi))

    # Export to TFRecord file
    task = ee.batch.Export.table.toDrive(
        collection=samples,
        description=PREFIX,
        folder=PREFIX,  
        fileNamePrefix=f'{year}_{longtitude}_{latitude}_{wi}',
        fileFormat='TFRecord'
    )
    return task

# Function to sample patch and include additional properties
def sample_patch(point, patches_array, longtitude, latitude, wi):
    arrays_samples = patches_array.sample(
        region=point.geometry(),
        scale=SCALE,
        projection='EPSG:3857',
        factor=None,
        numPixels=None,
        dropNulls=False,
        tileScale=12)
    sample_feature = arrays_samples.first()

    # Set additional properties to the feature
    sample_feature = sample_feature.set({
        'longtitude': longtitude,
        'latitude': latitude,
        'wi' : wi
    })
    return sample_feature

In [8]:
for i in range(NUM_BATCH):
    batch_df = df[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]
    print(i)
    for idx, row in batch_df.iterrows():
        longtitude = row['centroid_longtitude']
        latitude = row['centroid_latitude']
        year = row['year']
        wi = row['wi']
        try:
            export_image(longtitude, latitude, 2016 if year == 2014 else year, wi)
        except:
            print(f'Error at {longtitude}, {latitude}, {year}')
            time.sleep(30)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
